In [1]:
import numpy as np
import pandas as pd

from collab2.foraging.toolkit import dataObject, filter_by_distance, rescale_to_grid

In [2]:
# create a test object

num_frames = 8
num_foragers = 4
grid_size = 30
n_nans = int(num_frames * num_foragers / 5)
n_missing = int(num_frames * num_foragers / 5)
gridMin = -1
gridMax = 1

np.random.seed(42)

# generate data
theta = 2 * np.pi * np.random.rand(num_frames * num_foragers)
r = np.random.rand(num_frames * num_foragers)

data = {
    "x": r * np.cos(theta),
    "y": r * np.sin(theta),
    "time": np.tile(np.arange(num_frames), num_foragers),
    "forager": np.concatenate([i * np.ones(num_frames) for i in range(num_foragers)]),
}

foragersDF = pd.DataFrame(data)

# add nan values
nan_ind = np.random.randint(0, num_frames * num_foragers, size=n_nans)
foragersDF.loc[nan_ind, ["x", "y"]] = np.nan

# remove values for certain time points
drop_ind = np.random.randint(0, num_frames * num_foragers, size=n_missing)
foragersDF = foragersDF.drop(drop_ind)

# scaling and subsampling
foragersDF_scaled = rescale_to_grid(
    foragersDF, size=grid_size, gridMin=gridMin, gridMax=gridMax
)
foragers_object = dataObject(foragersDF_scaled, grid_size=grid_size)

display(foragers_object.foragers)

/Users/Admin/Repositories/collab-creatures/collab2/foraging/toolkit/utils.py:42: UserWarning: 
                NaN values in data. The default behavior of predictor/score generating functions is
                to ignore foragers with missing positional data. To modify, see documentation of
                `derive_predictors_and_scores` and `generate_local_windows`
                
  warnings.warn(
/Users/Admin/Repositories/collab-creatures/collab2/foraging/toolkit/utils.py:62: UserWarning: 
                    Missing frames encountered for forager 0, adding NaN fillers.
                    The default behavior of predictor/score generating functions is
                    to ignore foragers with missing positional data. To modify, see documentation of
                    `derive_predictors_and_scores` and `generate_local_windows`
                    
  warnings.warn(
/Users/Admin/Repositories/collab-creatures/collab2/foraging/toolkit/utils.py:62: UserWarning: 
                    Mis

[      x     y  time  forager
 0   NaN   NaN     0      0.0
 1  28.0  10.0     1      0.0
 2  13.0   0.0     2      0.0
 3   5.0   7.0     3      0.0
 4  17.0  18.0     4      0.0
 5  15.0  16.0     5      0.0
 6   NaN   NaN     6      0.0
 7  19.0  10.0     7      0.0,
       x     y  time  forager
 0   NaN   NaN     0      1.0
 1  13.0   7.0     1      1.0
 2  15.0  15.0     2      1.0
 3  28.0  12.0     3      1.0
 4   NaN   NaN     4      1.0
 5  17.0  24.0     5      1.0
 6   NaN   NaN     6      1.0
 7  18.0  22.0     7      1.0,
       x     y  time  forager
 0  12.0  22.0     0      2.0
 1  12.0  14.0     1      2.0
 2   1.0  21.0     2      2.0
 3  12.0  26.0     3      2.0
 4   4.0   5.0     4      2.0
 5  23.0  25.0     5      2.0
 6  12.0  23.0     6      2.0
 7   5.0  25.0     7      2.0,
       x     y  time  forager
 0   NaN   NaN     0      3.0
 1  15.0  12.0     1      3.0
 2  15.0  15.0     2      3.0
 3   NaN   NaN     3      3.0
 4   NaN   NaN     4      3.0
 5  18.

In [3]:
# interaction partners of forager f at time t
f = 2
t = 1
L = 15
filter_by_distance(foragers_object.foragersDF, f=f, t=t, interaction_length=L)

[1.0, 3.0]

In [4]:
# adding constraint

# suppose we only want foragers that are on the right half of the grid

# convention : every constraint function must take a list of foragers (within interaction distance), index of focal forager, current time, foragersDF + any additional parameters


def filter_x_constraint(f_ind, f, t, foragersDF, grid_size):
    current_positions = foragersDF.loc[
        np.logical_and(foragersDF["forager"].isin(f_ind), foragersDF["time"] == t)
    ]
    f_ind_constr = current_positions.loc[
        current_positions["x"] >= grid_size / 2, "forager"
    ].to_list()
    return f_ind_constr


# interaction partners of forager 1 at time 1 with constraint
filter_by_distance(
    foragers_object.foragersDF,
    f=f,
    t=t,
    interaction_length=L,
    interaction_constraint=filter_x_constraint,
    grid_size=foragers_object.grid_size,
)

[3.0]